In [41]:
import pandas as pd
import altair as alt
import warnings
import requests
import datetime
import matplotlib.pyplot as plt
import json
import requests
import numpy as np
alt.renderers.set_embed_options(theme='dark')
pd.set_option('display.max_colwidth', None)

In [205]:
class AstroDataProvider:
    
    def __init__(self, claim):
        daic_url = "https://terra-api.daic.capital/api/tx/GetRichlistByTokenContract?apiKey=vAp6ysmAXH470YcphYxv&contract_address={}"
        self.votes = '4940a215-6e93-4107-bf08-50574b3e431d'
        self.astro_holders_url =daic_url.format("terra1xj49zyqrwpv5k928jwfpfy2ha668nwdgkwlrg3")
        self.xastro_holders_url =daic_url.format("terra14lpnyzc9z4g3ugr4lhm8s4nle0tq8vcltkhzh7")
        self.claim = claim
        
    def get_from_url(self, url):
        json = requests.get(url).json()
        return json
        
    def load(self):
        self.votes_df = self.claim(self.votes)
        #
        json = self.get_from_url(self.astro_holders_url)['result']['holders']
        self.astro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.astro_holders_df.columns =  ['addr','amount']
        #
        json = self.get_from_url(self.xastro_holders_url)['result']['holders']
        self.xastro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.xastro_holders_df.columns =  ['addr','amount']
        #
        
    def parse_proposal_recap(self):
        votes = self.votes_df.groupby(['proposal_id','vote']).sum().voting_power.reset_index()
        against = votes[votes.vote=='against']
        against.columns = ['proposal_id','against','voting_power_against']
        for_ = votes[votes.vote=='for']
        for_.columns = ['proposal_id','for','voting_power_for']
        votes = against.merge(for_, on='proposal_id')
        votes['delta'] = votes['voting_power_for'] - votes['voting_power_against'] 
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        return votes
    
    def parse_top_active_voters(self):
        return dp.votes_df.groupby('voter').agg({'voting_power':'sum','tx_id':'count'})\
                            .sort_values(by=['tx_id','voting_power'], ascending=False)\
                            .head(20)
    def parse_dist_voting_power_per_proposal(self):
        return dp.votes_df[['proposal_id','voting_power']]\
            .pivot(columns='proposal_id',values='voting_power')
    
    def parse_top_voters_per_proposal(self):
        df=[]
        for i in range(1,8):
            if(len(df)==0):
                df = dp.votes_df[dp.votes_df.proposal_id==1].sort_values(by='voting_power', ascending=False).head(10)
            else:
                df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
        return df
    
    def parse(self):
        self.astro_holders_df.amount = self.astro_holders_df.amount/1000000
        self.proposal_recap = self.parse_proposal_recap()
        self.top_active_voters = self.parse_top_active_voters()
        self.dist_voting_power_per_proposal = self.parse_dist_voting_power_per_proposal()
        self.top_voters_per_proposal = self.parse_top_voters_per_proposal()
        
    def to_file(self, path='../data'):
        self.votes_df.to_json(f"{path}/votes_df",orient='records')
        self.astro_holders_df.to_json(f"{path}/astro_holders_df",orient='records')
        self.proposal_recap.to_json(f"{path}/proposal_recap", orient='records')
        self.top_active_voters.to_json(f"{path}/top_active_voters",orient='records')
        self.dist_voting_power_per_proposal.to_json(f"{path}/dist_voting_power_per_proposal",orient='records')
        self.top_voters_per_proposal.to_json(f"{path}/top_voters_per_proposal",orient='records')
        
    def read_file(self):
        url = 'https://raw.githubusercontent.com/IncioMan/astroport_governance/master/data/{}'
        self.votes_df =  pd.read_json(url.format('votes_df'))
        self.astro_holders_df =  pd.read_json(url.format('astro_holders_df'))
        self.proposal_recap =  pd.read_json(url.format('proposal_recap'))
        self.top_active_voters =  pd.read_json(url.format('top_active_voters'))
        self.dist_voting_power_per_proposal =  pd.read_json(url.format('dist_voting_power_per_proposal'))
        self.top_voters_per_proposal =  pd.read_json(url.format('top_voters_per_proposal'))


In [206]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [207]:
dp = AstroDataProvider(claim)
dp.load()
dp.parse()
dp.to_file()
dp.read_file()

/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_6757/2080283607.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_6757/2080283607.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_6757/2080283607.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
/var/folde

In [203]:
dp.votes_df[dp.votes_df.voter.fillna('0').str.endswith('kl60')]

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power
254,1651318804099,4C6CC1442ED56A61B00A15917E4DE60F54C785069D2965933D8CA4AC2DAF5451,cast_vote,6,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
2223,1649707102851,CA97CC66C576EC234AE0992CD4068DCA5A896E3409043E9E2BDE084ACFD88C10,cast_vote,1,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
4283,1651318762554,ADCDAB4895FD4D21A51475AD1182B7D7E6A9FB3FA2FB5DFC305602E679123493,cast_vote,7,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
4786,1650638346829,E796D2E0F43F0AED529F77C4736EFB6C0B3CDE35764B58F81E88A2FC45531DF3,cast_vote,2,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
5813,1650639524766,F4E213A2C112BF50D9946A6EAAEA36FF370FED9A56442E4A5AE344F7938FD5CA,cast_vote,5,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
6706,1650638771877,DB0EB3EDD43773AC4AE11EC5046947065E347C9E1F95A3D0060418810EB02E0A,cast_vote,3,against,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13
7272,1650639374675,0F8F4E3790D2285479A938C178BAD0958AFF6D80ED13814E9B32FFE56C61ACDA,cast_vote,4,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13


In [204]:
dp.top_voters_per_proposal[dp.top_voters_per_proposal.proposal_id==3]

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power
20,1650478432649,DCD5CD0A979B31A2BBD03782CE714C8E34E04A996D3DD000946F08896B8438E2,cast_vote,3,for,terra1l35xhpjnyqu86j6k59lqnw9mcavfr9vzhseyw7,27111
21,1650741371634,F542AEA57C2B4E9C021B1A3841E3EFB241B46B60E966AAD9098FA36395C84C0B,cast_vote,3,against,terra1sm8dmak34ay2acly50hwqjrmh56rle5msk0j27,39906
22,1650548584952,95DBBA7E16CB40CBF07251558A8C2919049E1BF2EC1E107403DA95BF1BB394B3,cast_vote,3,for,terra1ptyq4ggfdpy8egsju0xwx0q9zthh5sy0pncvyv,307244
23,1650515005132,C99F69BF511EBD66BC10FE77DCEF57617CE69681FE6ECBB2AAE3140052E65054,cast_vote,3,for,terra1c4z2qv69nlg9wk72aj9jeryay3uft0he3znxr2,395238
24,1650514456204,F58EC7F1994BAEC559CA7A9DA9359306519AF1C9C34FF6D6B9B32459850F4201,cast_vote,3,for,terra1w9339lh8pmtkjakeh9rwzg09hd6vvak3w5jsal,401948
25,1650513931291,C8CC0E26D619770CAB12DFC14FFC7916F26FDC1DE9B4F28FAEAD5D40087A8B82,cast_vote,3,for,terra1dn2jch6fwq5fecv9nsxa5rd7up56e0vknvs43m,434785
26,1650479228130,8384BB20E6F6882B8D480E38834421CD455FAAFC1CDBCC4B1748E59E1FA804EA,cast_vote,3,for,terra1ph6eqdgu5uvaqmnalf36fcwtp2q23krs6cyck7,446054
27,1650514748628,43DAE82DB01BA2D98D92AE6B6BB8C89C4A638169FF7F94FBF88DD550091A7E3A,cast_vote,3,for,terra19jdd2d0np8krkvurlfedpdxr8wfvu8vc3p9vrc,477376
28,1650479221415,CB03AAC8B1EE70522B33C9CA280722C529A8694752599B7345DCB172D4BEC882,cast_vote,3,for,terra10ldwt3gl5qs9a2ltkz3fmz50cgg5vyva9euvkt,704030
29,1650517073341,2863A01EAB0AAD74F7D0F9FAEAD17D86EBE0823F949431C51E864175F188DD4F,cast_vote,3,for,terra1xhd9wjcwx28sjktssu7u6kapu67g5t5tcaaqcm,866787


In [395]:
class NebulaChartProvider:
    
    def ust_traded_prices_chart(self, ust_traded_prices):
        chart = alt.Chart(ust_traded_prices).mark_point().encode(
        x=alt.X('Price:Q', sort=alt.EncodingSortField(order='ascending')),
        y="Amount UST (M):Q",
        color=alt.Color('Action:N', scale=alt.Scale(domain=['Sold NEB','Bought NEB'],
                                                      range=['#F24A72','#21bcd7'])),
        tooltip=['Action','Amount UST (M):N','Price:Q']
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_price_chart(self,df):
        cols = ['Number of Users','Price']
        chart = alt.Chart(df).mark_line(point=True).encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":Q",axis=alt.Axis(tickCount=20, labelAngle=0, tickBand = 'center')),
            tooltip=[cols[0],cols[1]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_time_chart(self,df):
        cols = ['Number of Users','Time'] 
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":T"),
            tooltip=[alt.Tooltip(cols[1]+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols[0]+":Q")]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def n_prices_per_users_df_chart(self,df):
        cols = ['Number of Users','Number of Different Prices']
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":N",axis=alt.Axis(tickCount=10, labelAngle=30, tickBand = 'center')),
            tooltip=[cols[1], cols[0]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def user_distr_pie(self, df, cols):
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
            theta=alt.Theta(field=cols[0], type="quantitative"),
            color=alt.Color(field=cols[1], type="nominal",
                    #sort=['MARS & UST','MARS','UST'],
                    scale=alt.Scale(domain=df[cols[1]].unique(), range=['#F24A72','#21bcd7']),
                    legend=alt.Legend(
                    orient='none',
                    padding=10,
                    legendY=-10,
                    direction='vertical')),
            tooltip=[cols[1]+':N',cols[0]+':N']
        ).configure_view(strokeOpacity=0)
        return chart
    
    def sender_airdrop_op_charts(self, df, cols):
        df.columns = cols
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
                    theta=alt.Theta(field=cols[1], type="quantitative"),
                    color=alt.Color(field=cols[0], type="nominal",
                            #sort=['MARS & UST','MARS','UST'],
                            scale=alt.Scale(domain=df[cols[0]].unique(), range=['#ffffff','#21bcd7','#F24A72']),
                            legend=alt.Legend(
                            orient='none',
                            padding=10,
                            legendY=-10,
                            direction='vertical')),
                    tooltip=[cols[1]+':N',cols[0]+':N']
                ).configure_view(strokeOpacity=0)
        return chart
    
    def price_chart(self,hourly_stats_df):
        #272231 background
        df=hourly_stats_df[['avg_belief_price','time']]
        df.columns=['Price','Hour']
        n_data = 20
        if df.Hour.nunique() < n_data:
            extra_data = []
            for i in range(n_data-df.Hour.nunique()):
                extra_data.append([None,(pd.to_datetime(df.Hour.max())+datetime.timedelta(hours=i)).strftime("%Y-%m-%d %H:%M")])
            df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))
        else:
            df2 = df
        chart = alt.Chart(df2).mark_line(point=True).encode(
            x=alt.X('Hour:T', sort=alt.EncodingSortField(order='ascending')),
            y="Price:Q",
            tooltip=['Hour:T',"Price:Q"]
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart